# Lab Exercise 1. Scraping Static Websites


This is the warmup task for the first laboratory exercise. It consists of scraping static Websites with BeautifulSoap.

 It should be completed at home and presented at the laboratory.

**Total points: 2**

### Task Description

Scrape the information about the products on the following page:
https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/

For each product scrape:


*   Product title (selector `'.wd-entities-title'`)
*   Product regular price (selector `'.woocommerce-Price-amount'`)
*   Product discount price (if available), same selector as regular price
*   URL to the product page
*   Add to cart button URL

***Help: There are multiple product pages, for each page you need to send a separate request***


Save the results as a DataFrame object

You can add as many code cells as you need.

________________________________________________________________

### Requirements

Import libraries and modules that you are going to use

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Send HTTP request to the target Website

In [42]:
response = requests.get("https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/")
response

<Response [200]>

check the response status code

In [43]:
response.status_code

200

### Parse the response content with BeautifulSoap

In [44]:
soup = BeautifulSoup(response.text, "html.parser")

### Extract data from the BeautifulSoap object using any selectors, attribute identifiers, etc.

* Product title (selector '.wd-entities-title')
* Product regular price (selector '.woocommerce-Price-amount')
* Product discount price (if available), same selector as regular price
* URL to the product page
* Add to cart button URL

In [45]:
title= soup.select('.wd-entities-title')
title[:5]

[<a class="wd-entities-title" href="https://clevershop.mk/product/oukitel-wp20-pro/" title="Oukitel WP20 Pro">
 Oukitel WP20 Pro			</a>,
 <a class="wd-entities-title" href="https://clevershop.mk/product/oukitel-wp20/" title="Oukitel WP20">
 Oukitel WP20			</a>,
 <h3 class="wd-entities-title"><a href="https://clevershop.mk/product/acer-a315-23-a7kd/">Acer A315-23-A7KD</a></h3>,
 <h3 class="wd-entities-title"><a href="https://clevershop.mk/product/acer-a315-23-r5p2/">Acer A315-23-R5P2</a></h3>,
 <h3 class="wd-entities-title"><a href="https://clevershop.mk/product/acer-aspire-1-nx-a7pex-001/">ACER Aspire 1 A115-22</a></h3>]

In [30]:
price = soup.select('.woocommerce-Price-amount')
price[:5]

[<span class="woocommerce-Price-amount amount"><bdi>0 <span class="woocommerce-Price-currencySymbol">ден</span></bdi></span>,
 <span class="woocommerce-Price-amount amount"><bdi>0 <span class="woocommerce-Price-currencySymbol">ден</span></bdi></span>,
 <span class="woocommerce-Price-amount amount"><bdi>9.890 <span class="woocommerce-Price-currencySymbol">ден</span></bdi></span>,
 <span class="woocommerce-Price-amount amount"><bdi>8.490 <span class="woocommerce-Price-currencySymbol">ден</span></bdi></span>,
 <span class="woocommerce-Price-amount amount"><bdi>17.590 <span class="woocommerce-Price-currencySymbol">ден</span></bdi></span>]

In [36]:
discount = soup.select('.woocommerce-Price-amount')
discount_prices = discount[:5] if len(discount) >= 3 else []

In [35]:
links = [a.get('href') for a in soup.select('a')]
links[:5]

['https://www.facebook.com/sharer/sharer.php?u=https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/',
 'https://twitter.com/share?url=https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/',
 'https://pinterest.com/pin/create/button/?url=https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/&media=https://clevershop.mk/wp-content/uploads/2022/08/acer-a315-23-a7kd-nxhvuex037_62ae121eb61d9.png&description=Acer+A315-23-A7KD',
 'https://www.linkedin.com/shareArticle?mini=true&url=https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/',
 'https://telegram.me/share/url?url=https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/']

In [39]:
buttons = [a.get('href') for a in soup.select('a.button') if a.get('href')]
buttons[:5]

['?add-to-cart=21494',
 '?add-to-cart=21510',
 '?add-to-cart=20826',
 '?add-to-cart=21516',
 '?add-to-cart=21498']

Repeat the extraction process for each page of products

In [62]:
base_url = "https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/"

def multiple_pages(product):
    title = product.select_one('.wd-entities-title').text.strip() if product.select_one('.wd-entities-title') else None
    price = product.select_one('.woocommerce-Price-amount').text.strip() if product.select_one('.woocommerce-Price-amount') else None
    link = product.select_one('a')['href'] if product.select_one('a') else None
    button = product.select_one('a.button')['href'] if product.select_one('a.button') else None

    product_dict = {
        "ProductTitle": title,
        "Price": price,
        "Link": link,
        "AddToCartButton": button
    }

    return product_dict

all_products = []
for i in range(1, 14):
    url = f"{base_url}{i}/"
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        products = soup.select('.product')

        if products:
            for product in products:
                result = multiple_pages(product)
                all_products.append(result)
        else:
            print(f"No products found on page {i}.")
    else:
        print(f"Failed to retrieve page {i}. Status code: {response.status_code}")

### Create a pandas DataFrame with the scraped products

In [64]:
final = pd.DataFrame(all_products)
final.head()

,ProductTitle,Price,Link,AddToCartButton
0,Acer A315-23-A7KD,17.590 ден,https://clevershop.mk/product/acer-a315-23-a7kd/,?add-to-cart=21494
1,Acer A315-23-R5P2,27.490 ден,https://clevershop.mk/product/acer-a315-23-r5p2/,?add-to-cart=21510
2,ACER Aspire 1 A115-22,18.999 ден,https://clevershop.mk/product/acer-aspire-1-nx...,?add-to-cart=20826
3,Acer Aspire 3 A315-23-R26A,29.990 ден,https://clevershop.mk/product/acer-aspire-3-a3...,?add-to-cart=21516
4,Acer Aspire 3 A315-58-33WK,24.490 ден,https://clevershop.mk/product/21498/,?add-to-cart=21498


Save the dataframe as `.csv`

In [65]:
final.to_csv('all_products_csvfile.csv', index=False)